# Imports, define paths

In [ ]:
import os
import numpy as np
import helper_funcs as hf

In [ ]:
geom = 'p4_square_2024-05-22_15-01-53.190117'
# geom = 'p4m_square_2024-05-22_15-08-35.472754'
# geom = 'p6_hexagonal_2024-05-22_15-52-16.388326'
# geom = 'pg_rectangular_2024-05-22_14-21-22.169238'

# specify path to folder for this geometry
path = os.path.join(r'data', geom)

In [ ]:
geo_file = os.path.join(path, geom + '_00.geo')
print(geo_file)

Original hmax = 0.1/sqrt(6) = 0.040824829046386304


In [ ]:
# define the path to the directory where the mesh files will be saved
save_dir = r'mesh_convergence_' + geom

# create the directory if it does not exist
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Create .geo files

In [ ]:
# create .geo files with different hmax values
hmax_arr = 2.0**np.arange(-8, -1.9, 0.5)
for hmax in hmax_arr:
    print(hmax)
    # import the original .geo file:
    with open(geo_file, 'r') as file:
        data = file.readlines()

    # replace the hmax value:
    for i, line in enumerate(data):
        if line.startswith('Point('):
            data[i] = line.rsplit(',', maxsplit=1)[0] + ', ' + str(hmax) + '};\n'
    data.extend([f'Mesh.CharacteristicLengthMax = {hmax};\n',
                 f'Mesh.CharacteristicLengthMin = {hmax};\n'])
    print(data)

    # write the new .geo file:
    with open(os.path.join(save_dir, f'hmax_{hmax}.geo'), 'w') as file:
        file.writelines(data)

# Create meshes and MatLab inputs

In [ ]:
import meshio
import subprocess
import pickle
import matplotlib.pyplot as plt
import time
import scipy
import copy

fig_nr = 0
verbose = True
figures = 0  # create lots of figuers

group = geom.split('_')[0]
print('Group:', group)

In [ ]:

def import_gmsh_mesh(file_path, element_type):
    # Load the Gmsh mesh from the .msh file
    mesh = meshio.read(file_path)

    # Access mesh information
    points = mesh.points[:, :2]
    cells = mesh.cells
    cell_data = mesh.cell_data

    elements = np.array([], dtype=int).reshape(0, 6)
    for cell in cells:
        if cell.type == element_type:
            elements = np.append(elements, cell.data, axis=0)

    return points, cells, cell_data, elements

Needed: .mat file with
* p
* t
* lattice_vectors
* boundary_inds

In [ ]:
files = os.listdir(save_dir)
for file in files:  #['hmax_0.00390625.geo']:  #
    try:
        if not file.endswith('.geo'):
            continue

        start_time = time.time()
        time_temp = time.time()
        file_path = os.path.join(save_dir, file)
        print(file_path)
        print(file)
        name = file[:-4]
        print(name)

        hmax = name.split('_')[1]
        print('hmax:', hmax)

    # %% [markdown]
        # Run gmsh on created .geo file with
        # Command example: 'gmsh', file, '-o', file2, '-2', '-clmax', str(hmax)]
        # ['gmsh', 'test.geo', '-o', 'test.msh', '-2', '-clmax', '0.1']
        # Explanation: gmsh: program to call, should be in path
        # file: path to input file
        # -o: specifying output file
        # file2: path to output file
        # -2: apparently indicates you want to use the command line interface of gmsh, without this it will open the gui
        # -clmax: specifying max element size
        # str(hmax): max element size

        # create new file path with .msh instead of .geo
        file2 = os.path.join(save_dir, f'{name}.msh')

        if verbose:
            print('Calling gmsh...')
        command = f'gmsh "{file_path}" -o "{file2}" -2 -clmax {str(hmax)}'
        print(f'command: {command}')
        process = subprocess.call(command, shell=True)

        if verbose:
            print('succesfully ran gmsh')
            print('Time for gmsh:', time.time()-time_temp)
            time_temp = time.time()
            # print(process.stdout.read())

        mesh = meshio.read(file2)
        if verbose:
            # print(mesh)
            pass

        # %%
        # ## Import mesh



        # %%
        mesh_points, _, _, elements = import_gmsh_mesh(file2, 'triangle6')

        # with open(hf.new_path(os.path.join(save_dir, f'{name}_fd.pkl')), 'wb') as f:
        #     pickle.dump({'p': mesh_points, 't': elements}, f)

        with open(os.path.join(path, f'{geom}_00.pkl'), 'rb') as f:
            data = pickle.load(f)
            uc = data['uc']
            fd = data['fd']
            unique_holes = data['unique_holes']

        # %%
        # Remove unused nodes
        to_keep, inv = np.unique(elements, return_inverse=True)
        mesh_points = mesh_points[to_keep]

        # renumber elements
        temp = np.full(elements.max()+1, -1, dtype=int)
        temp[to_keep] = np.arange(len(to_keep))
        elements = temp[elements]

        # %%
        # Check volume fraction again, this time of the actual mesh

        temp = mesh_points[elements]
        # temp = np.transpose(temp, axes=[0,2,1])
        temp = temp[..., [0,3,1,4,2,5], :]
        filled_area = np.sum(hf.polygon_area(temp))
        total_area = 1.0/uc['n_fds']
        vol_frac2 = filled_area/total_area

        if verbose:
            print(f'Volume occupied = {vol_frac2*100:.1f}%')
            print(f'Volume vacant = {(1-vol_frac2)*100:.1f}%')

            print('Time for processing fundamental domain mesh:', time.time()-time_temp)
            time_temp = time.time()

        # %% Turn fundamental domain into unit cell
        all_mesh_points = []
        all_elements = []
        inds_per_fd = []
        n = len(mesh_points)
        for j, copy1 in enumerate(uc['transforms']):

            p2 = np.copy(mesh_points)

            for transform in copy1:

                if transform[0] == 'T':
                    p2 = hf.translate_points(p2, eval(transform[1]))
                elif transform[0] == 'R':
                    degrees = np.array(eval(transform[2]))
                    p2 = hf.rotate_points(p2, eval(transform[1]), degrees/360*2*np.pi)
                elif transform[0] == 'M':
                    p2 = hf.mirror_points(p2, eval(transform[1]), eval(transform[2]))
                else:
                    raise NotImplementedError(f'transform {transform[0]} is not implemented')

            all_mesh_points.append(np.copy(p2))
            all_elements.append(np.copy(elements) + j*n)
            inds_per_fd.append(np.arange(n) + j*n)

        all_mesh_points = np.concatenate(all_mesh_points, axis=0)
        all_elements = np.concatenate(all_elements, axis=0)
        inds_per_fd = np.array(inds_per_fd)
        if verbose:
            print('all_mesh_points.shape', all_mesh_points.shape)
            print('all_elements.shape', all_elements.shape)
            print('inds_per_fd.shape', inds_per_fd.shape)

        # %% Deduplicate points
        start_time_dedup = time.time()
        # deduplicate
        all_mesh_points, inds, inv, c = hf.uniquetol(all_mesh_points, tol=1e-4, return_counts=True, return_index=True, return_inverse=True, axis=0)
        if verbose:
            print(f'Time for deduplication: {time.time() - start_time_dedup:.4} seconds')
            print('Time for tiling into unit cell:', time.time()-time_temp)
            time_temp = time.time()
        all_elements = inv[all_elements]
        inds_per_fd = inv[inds_per_fd]

        # %%
        # Plot counts
        if figures == 2:
            fig = plt.figure(figsize=(10,10))
            temp = all_mesh_points[all_elements]
            temp = np.transpose(temp, axes=[0,2,1])
            temp = temp[..., [0,3,1,4,2,5]]
            temp = temp.reshape(-1, temp.shape[-1])
            plt.fill(*temp, c='whitesmoke')
            plt.title('counts per point, should be =/= 1 at and only at overlapping fundamental domain boundaries')

            for count in np.unique(c):
                plt.scatter(*all_mesh_points[c == count].T, s=count*5, zorder=10, label=count)

            plt.gca().set_aspect('equal')
            plt.legend(title='count')

            path1 = hf.new_path(os.path.join(save_dir, f'fig_{fig_nr}_counts.png'))
            fig_nr += 1
            plt.axis('off')
            fig.savefig(path1)
            plt.close()

        # %%
        # Plot unit cell mesh
        if figures in [1,2]:
            fig = plt.figure(figsize=(10,10))
            fig.patch.set_facecolor("None")

            # background fundamental domain shape
            plt.fill(*uc["corners"].T, c='whitesmoke')

            # background unit cell shape
            plt.fill(*fd['corners'].T, c='lightgrey')  # c='tab:orange')

            # plot filled triangles
            temp = all_mesh_points[all_elements]
            temp = np.transpose(temp, axes=[0,2,1])
            temp = temp.reshape(-1, temp.shape[-1])
            temp = temp[..., [0,3,1,4,2,5]]  # order the nodes counterclockwise
            plt.fill(*temp, alpha=0.5)  # c='whitesmoke')

            # plot points
            plt.scatter(*all_mesh_points.T, s=1, zorder=10, c='black')

            plt.gca().set_aspect('equal')

            path1 = hf.new_path(os.path.join(save_dir, f'fig_{fig_nr}_unit_cell_mesh.png'))
            fig_nr += 1
            plt.axis('off')
            fig.savefig(path1)

            plt.close()

        # %%
        # Plot unit cell 2×2
        if figures in [1,2]:
            fig, ax = plt.subplots(figsize=(7,7))
            # ax.set_title(f'{group} ({shape})')
            fig.patch.set_facecolor("None")

            # background fundamental domain shape
            ax.fill(*uc["corners"].T, c='whitesmoke', zorder=-11)

            # background fundamental domain shape
            ax.fill(*fd['corners'].T, c='lightgrey', zorder=-10)

            vecs = uc['lattice vectors']

            # loop over vecs[0] translations
            for i in range(2):
                # loop over vecs[1] translations
                for j in range(2):
                    points_temp = hf.translate_points(all_mesh_points, i*vecs[0]+j*vecs[1])

                    # plot filled triangles
                    temp = points_temp[all_elements]
                    temp = np.transpose(temp, axes=[0,2,1])
                    temp = temp.reshape(-1, temp.shape[-1])
                    temp = temp[..., [0,3,1,4,2,5]]
                    if i == 0 and j == 0:
                        plt.fill(*temp)  #, alpha=0.5)
                    else:
                        plt.fill(*temp, c='tab:orange')

                    # ax.scatter(*points_temp.T, alpha=0.5, s=50)  #, c='tab:orange')
                    # x, y = np.transpose(points_temp[uc["edges"].T], axes=[2,0,1])
                    # edges0 = ax.plot(x, y, alpha=0.3, c='tab:red')

            ax.set_aspect('equal')

            path1 = hf.new_path(os.path.join(save_dir, f'fig_{fig_nr}_RVE.png'))
            fig_nr += 1
            plt.axis('off')
            fig.savefig(path1)
            plt.close()

        if verbose:
            print('Time for plots:', time.time()-time_temp)
            time_temp = time.time()

        # %%
        tol = 1e-5

        # find new boundary nodes
        mesh_bounds = []
        for b_start, b_vec in uc['bounds']:
            mesh_bounds.append([])
            bools_onbound = hf.iscollinear(all_mesh_points, b_start, b_start+b_vec, tol_g=1e-5)
            mesh_bounds[-1] = np.where(bools_onbound)[0]

            # sort them in the correct order
            if np.abs(b_vec[0]) > tol:
                temp = np.argsort(all_mesh_points[mesh_bounds[-1], 0]/b_vec[0])
            elif np.abs(b_vec[1]) > tol:
                temp = np.argsort(all_mesh_points[mesh_bounds[-1], 1]/b_vec[1])
            else:
                raise ValueError(f"bad value of b_vec: {b_vec}")
            mesh_bounds[-1] = mesh_bounds[-1][temp].tolist()

        mesh_bounds

        # %% Plot to check boundary nodes
        if figures == 2:
            fig, ax = plt.subplots(figsize=(7,7))
            # ax.set_title(f'{group} ({shape})')
            fig.patch.set_facecolor("None")

            plt.scatter(*all_mesh_points.T, alpha=0.3, s=5)

            temp = all_elements[:, [0,3,1,4,2,5,0]]
            x, y = np.transpose(all_mesh_points[temp.T], axes=[2,0,1])
            edges0 = plt.plot(x, y, alpha=0.1, c='tab:red', zorder=-1)

            for inds in mesh_bounds:
                if verbose:
                    print(len(inds))
                plt.scatter(*all_mesh_points[inds].T, s=5)

            plt.gca().set_aspect('equal')

            path1 = hf.new_path(os.path.join(save_dir, f'fig_{fig_nr}_check_boundary_nodes.png'))
            fig_nr += 1
            plt.axis('off')
            fig.savefig(path1)
            plt.close()

        if verbose:
            print('Time for finding and plotting boundary nodes:', time.time()-time_temp)
            time_temp = time.time()

        # %% flip mirrored elements
        temp = all_mesh_points[all_elements]
        temp = temp[..., [0,3,1,4,2,5], :]
        signed_areas = hf.polygon_area(temp, signed=True)
        print("all_elements[signed_areas < 0].shape:")
        print(all_elements[signed_areas < 0].shape)
        all_elements[signed_areas < 0] = all_elements[signed_areas < 0][:, [2,1,0,4,3,5]]

        if verbose:
            print('Time to find and flip mirrored elements:', time.time()-time_temp)
            time_temp = time.time()

        # %% Save things
        for i in range(len(unique_holes)):
            try:
                unique_holes[i]['poly'] = unique_holes[i]['poly'].coords
            except AttributeError as e:
                if verbose:
                    print(repr(e))

        for i in range(len(unique_holes)):
            try:
                del unique_holes[i]['skel']
            except KeyError as e:
                if verbose:
                    print(repr(e))

        with open(hf.new_path(os.path.join(save_dir, f'{name}.pkl')), 'wb') as f:
            pickle.dump({'uc': uc, 'fd': fd,
                        'p': mesh_points, 't': elements,
                        'p_all': all_mesh_points, 't_all': all_elements,
                        'boundary_inds': mesh_bounds,
                        'unique_holes': unique_holes,
                    }, f)

        if verbose:
            print(f'Time for saving: {time.time() - time_temp:.4} seconds')
            time_temp = time.time()

        # %% [markdown]
        # turn unit cell into RVE: tile 2×2
        all_mesh_points2 = np.concatenate((all_mesh_points,
                                    all_mesh_points + uc['lattice vectors'][0],
                                    all_mesh_points + uc['lattice vectors'][1],
                                    all_mesh_points + uc['lattice vectors'][0] + uc['lattice vectors'][1],
                                    ), axis=0)
        n = len(all_mesh_points)
        all_elements2 = np.concatenate((all_elements,
                                        all_elements + n,
                                        all_elements + 2*n,
                                        all_elements + 3*n,
                                        ), axis=0)

        inds_per_fd2 = np.tile(inds_per_fd, reps=(4, 1, 1))
        inds_per_fd2 += np.arange(4).reshape(-1, 1, 1)*n
        if verbose:
            print('inds_per_fd2.shape', inds_per_fd2.shape)

        mesh_bounds2 = [mesh_bounds,
                        [(np.array(mb) + n).tolist() for mb in mesh_bounds],
                        [(np.array(mb) + 2*n).tolist() for mb in mesh_bounds],
                        [(np.array(mb) + 3*n).tolist() for mb in mesh_bounds],]

        # %% Deduplicate (again)
        start_time_dedup = time.time()
        all_mesh_points4, inds, inv, c = hf.uniquetol(all_mesh_points2, tol=1e-5, return_counts=True, return_index=True, return_inverse=True, axis=0)
        if verbose:
            print(f'Time for deduplication: {time.time() - start_time_dedup:.4} seconds')
        all_elements4 = inv[all_elements2]
        inds_per_fd4 = inv[inds_per_fd2]

        for i in range(len(mesh_bounds2)):
            for j in range(len(mesh_bounds2[i])):
                mesh_bounds2[i][j] = inv[mesh_bounds2[i][j]]

        if verbose:
            print('Time to tile into RVE:', time.time()-time_temp)
            time_temp = time.time()

        # %%
        # Plot RVE, with unit cell boundaries
        if figures == 2:
            fig, ax = plt.subplots(figsize=(7,7))
            # ax.set_title(f'{group} ({shape})')
            fig.patch.set_facecolor("None")

            # Plot after deduplication
            temp = all_mesh_points4[all_elements4]
            temp = np.transpose(temp, axes=[0,2,1])
            temp = temp[..., [0,3,1,4,2,5]]
            temp = temp.reshape(-1, temp.shape[-1])
            plt.fill(*temp, alpha=0.5)  #, c='tab:orange')

            for mb in mesh_bounds2:
                for mb2 in mb:
                    plt.scatter(*all_mesh_points4[mb2].T, s=5, zorder=11)

            plt.scatter(*all_mesh_points4[inds_per_fd4[0][0]].T, s=1, zorder=10, c='black')
            plt.scatter(*all_mesh_points4[inds_per_fd4[1][0]].T, s=1, zorder=10, c='black')

            # plt.scatter(*all_mesh_points4.T, s=1, zorder=10)
            plt.gca().set_aspect('equal')

            path1 = hf.new_path(os.path.join(save_dir, f'fig_{fig_nr}_RVE_with_uc_boundaries.png'))
            fig_nr += 1
            plt.axis('off')
            fig.savefig(path1)
            plt.close()

        # %%
        if figures == 2:
            # Plot counts
            fig, ax = plt.subplots(figsize=(10,10))
            # ax.set_title(f'{group} ({shape})')
            fig.patch.set_facecolor("None")

            temp = all_mesh_points4[all_elements4]
            temp = np.transpose(temp, axes=[0,2,1])
            temp = temp[..., [0,3,1,4,2,5]]
            temp = temp.reshape(-1, temp.shape[-1])
            plt.fill(*temp, c='whitesmoke')
            plt.title('counts per point, should be =/= 1 at and only at overlapping unit cell boundaries')

            for count in np.unique(c):
                plt.scatter(*all_mesh_points4[c == count].T, s=count*5, zorder=10, label=count)

            plt.gca().set_aspect('equal')
            plt.legend(title='count', loc=1)

            path1 = hf.new_path(os.path.join(save_dir, f'fig_{fig_nr}_counts.png'))
            fig_nr += 1
            plt.axis('off')
            fig.savefig(path1)
            plt.close()

        if verbose:
            print('Time for plots:', time.time()-time_temp)
            time_temp = time.time()

        # %%
        # meshbounds: find new boundary nodes of RVE (old boundary points and copies thereof, minus the deduplicated ones)
        mesh_bounds4 = [[] for asdf in mesh_bounds2[0]]

        # iterate over unit cells
        for i in range(len(mesh_bounds2)):
            # iterate over direction of boundary
            # oblique: bottom, right, top, left
            # hexagonal: bottom, bottomright, topright, top, topleft, bottomleft
            for j in range(len(mesh_bounds2[i])):
                # check if it's an internal boundary by checking if all of the nodes had duplicates
                if (c[mesh_bounds2[i][j]] == 1).any():
                    mesh_bounds4[j].extend(mesh_bounds2[i][j])

        # %%
        # Check if number of boundary nodes is consistent with the unit cell
        for i in range(len(mesh_bounds4)):
            mesh_bounds4[i] = np.unique(mesh_bounds4[i]).tolist()
            if hf.wallpaper_groups[group]['unit cell shape'] == 'hexagon':
                len_new = len(mesh_bounds4[i])
                len_old = len(mesh_bounds2[0][i])
                assert 2*len_old-2 <= len_new <= 3*len_old, 'boundaries are the wrong size!'
            elif hf.wallpaper_groups[group]['unit cell shape'] == 'parallelogram':
                len_new = len(mesh_bounds4[i])
                len_old = len(mesh_bounds2[0][i])
                assert 2*len_old-2 <= len_new <= 2*len_old, 'boundaries are the wrong size!'
            else:
                raise ValueError(f"shape {hf.wallpaper_groups[group]['unit cell shape'] } not implemented")

        # %%
        # sort boundary nodes in the correct order
        for i, mb in enumerate(mesh_bounds4):
            coords = all_mesh_points4[mb]

            # sort by whichever has a larger difference: x- or y-coordinate
            x_diff = np.max(coords[:, 0]) - np.min(coords[:,0])
            y_diff = np.max(coords[:, 1]) - np.min(coords[:,1])

            if group == 'p3' or group == 'p3m1':
                temp2 = np.einsum('i,ji->j', uc['bounds'][i][1], coords)
                mesh_bounds4[i] = np.array(mesh_bounds4[i])[np.argsort(temp2)]
                if i >= 3:
                    mesh_bounds4[i] = np.flip(mesh_bounds4[i], axis=0)
            else:
                if x_diff > y_diff:
                    mesh_bounds4[i] = np.array(mesh_bounds4[i])[np.argsort(coords[:, 0])]
                # else sort by y-coordinate
                else:
                    mesh_bounds4[i] = np.array(mesh_bounds4[i])[np.argsort(coords[:, 1])]

        # in the case of a hexagonal unit cell, the third boundary has a different order for the source vs the image nodes (reverse order of thirds)
        if group == 'p3':
            n = len(mesh_bounds4[3])//3
            mesh_bounds4[3] = np.concatenate((mesh_bounds4[3][2*n:],
                                            mesh_bounds4[3][n:2*n],
                                            mesh_bounds4[3][:n]), axis=0)

        if group == 'p3m1':
            n = len(mesh_bounds4[5])//3
            mesh_bounds4[5] = np.concatenate((mesh_bounds4[5][2*n:],
                                            mesh_bounds4[5][n:2*n],
                                            mesh_bounds4[5][:n]), axis=0)

        if verbose:
            print('Time to get RVE boundaries:', time.time()-time_temp)
            time_temp = time.time()

        # %% Plot to check boundaries
        if figures == 2:
            fig, ax = plt.subplots(figsize=(10,10))
            # ax.set_title(f'{group} ({shape})')
            fig.patch.set_facecolor("None")

            temp = all_mesh_points4[all_elements4]
            temp = np.transpose(temp, axes=[0,2,1])
            temp = temp[..., [0,3,1,4,2,5]]
            temp = temp.reshape(-1, temp.shape[-1])
            plt.fill(*temp, c='whitesmoke')

            for mb in mesh_bounds4:
                plt.scatter(*all_mesh_points4[mb].T, s=5, zorder=11, alpha=0.5)
                plt.scatter(*all_mesh_points4[mb][:10].T, s=2, zorder=11)

            plt.scatter(*all_mesh_points4.T, s=1, zorder=10, c='black')
            plt.gca().set_aspect('equal')
            plt.title('Check boundaries')

            path1 = hf.new_path(os.path.join(save_dir, f'fig_{fig_nr}_check_boundaries.png'))
            fig_nr += 1
            plt.axis('off')
            fig.savefig(path1)
            plt.close()

        if verbose:
            print('Time for plotting boundaries:', time.time()-time_temp)
            time_temp = time.time()

        print('create & deduplicate edges for crossing edges check')
        t = all_elements4
        edges = np.vstack((t[:, [0, 3]],
                    t[:, [3, 1]],
                    t[:, [1, 4]],
                    t[:, [4, 2]],
                    t[:, [2, 5]],
                    t[:, [5, 0]]))

        edges2 = np.sort(edges, axis=-1)
        edges3, inv, counts = np.unique(edges2, axis=0, return_inverse=True, return_counts=True)

        if np.any(counts > 2):
            raise ValueError('Some edges are used more than twice')

        if verbose:
            print('Time to create and deduplicate edges:', time.time()-time_temp)
            time_temp = time.time()

        # %% save to matlab stuff
        # needed:
        # * nodes: xyz coordinates of the nodes, shape [nr of nodes, 3] (z-coordinate can be zero)
        # * nNodes: nr of nodes (integer)
        # elements: indices of nodes defining elements, shape [nr of elements, 6], elements 0, 1, 2 are the corner nodes in counterclockwise order, 3,4,5 are the mid-edge nodes also in counterclockwise order
        # * elemMats: material of each element (all the same, can all be 1), shape [nr of elements, 1]
        # * nelems: nr of elements
        # FE2: 1×1 struct, contains:
        #   FE2.V: volume of RVE
        #   FE2.periodicSourceNodes: list of independent boundary + corner nodes (?)
        #   FE2.periodicImageNodes: list of corresponding dependent boundary + corner nodes
        # which nodes correspond to which fundamental domain

        transforms = copy.deepcopy(uc['transforms'])
        # to do: transforms to numerical form
        for i, temp in enumerate(uc['transforms']):
            for j, temp2 in enumerate(temp):
                for k, temp3 in enumerate(temp2):
                    try:
                        transforms[i][j][k] = eval(temp3)
                    except NameError:
                        transforms[i][j][k] = temp3
        if verbose:
            print(transforms)

        scipy.io.savemat(hf.new_path(os.path.join(save_dir, f'{name}.mat')),
                        {'p': all_mesh_points4,
                        't': all_elements4,
                        'boundary_inds': mesh_bounds4,
                        'inds_per_fd': inds_per_fd4,
                        'lattice_vectors': uc['lattice vectors'],
                        'transforms': transforms,
                        'a1': fd['a1'],
                        'a2': fd['a2'],
                        'volume_fraction': vol_frac2,
                        })
        print('Saved to .mat file')

        # %%
        print('Volume fraction based on mesh:', f'{vol_frac2*100:.1f}%')
        print(f'Total time to generate new material: {time.time()-start_time:.4} seconds')

        print('Points shape:', all_mesh_points4.shape)
        print('Elements shape:', all_elements4.shape)

        with open(hf.new_path(os.path.join(save_dir, 'info.txt')), 'w') as f:
            f.write(f'Volume fraction: {vol_frac2*100:.1f}%\n')
            f.write(f'Total time: {time.time()-start_time:.4} seconds\n')

        if verbose:
            print('Time for saving:', time.time()-time_temp)
            time_temp = time.time()

    except Exception as e:
        print(f'Error in {file}: {repr(e)}')
        continue

Use the meshes generated above in the matlab code.

# Import results
After the matlab code has been used to perform a simulation using each of the mesh files.

In [ ]:
import pickle
import os

import scipy.io as sio
import matplotlib.pyplot as plt
import numpy as np

import helper_funcs as hf

import scipy

In [ ]:
geom_base = 'p4_square_2024-05-22_15-01-53.190117'
# geom_base = 'p4m_square_2024-05-22_15-08-35.472754'
# geom_base = 'p6_hexagonal_2024-05-22_15-52-16.388326'
# geom_base = 'pg_rectangular_2024-05-22_14-21-22.169238'

results_path = r'path_to_matlab_results' + geom_base

# path where the meshes were saved
geometries_path = r'mesh_convergence_' + geom_base

In [ ]:
geoms = []
for file in os.listdir(results_path):
    if file.endswith('.mat') and not file.endswith('_specialnodes.mat'):
        geoms.append(file[:-4])

In [ ]:
# sort by hmax

hmax = np.array([float(geom.split('_')[1]) for geom in geoms])

inds = np.argsort(-hmax)

hmax = np.array(hmax)[inds]
geoms = np.array(geoms)[inds]

print(hmax)

In [ ]:
# keep only 0.00781 <= hmax < 0.25
inds = np.logical_and(hmax >= 0.00781, hmax < 0.25)  # 0.00781
hmax = hmax[inds]
geoms = geoms[inds]

In [ ]:
errors = []
rerun_found = False
data_all = []
for g, geom in enumerate(geoms):
    print(f'======= {g}, {geom} =======')
    # ====================== LOAD SIMULATION DATA ======================
    data = {'simulations': {},
            'time_steps': {},
            'geometry': {'fundamental_domain': {}, 'unit_cell': {}},
            'mesh': {'fundamental_domain': {}, 'unit_cell': {}, 'RVE': {}}
            }
    matfile = os.path.join(results_path, geom + '.mat')
    data_from_mat = sio.loadmat(matfile)
    for key in data_from_mat['data_sim'].dtype.names:
        data['simulations'][key] = data_from_mat['data_sim'][key][0, 0]
    for key in data_from_mat['data_ts'].dtype.names:
        data['time_steps'][key] = data_from_mat['data_ts'][key][0, 0]
    print('errorFlag:', data['simulations']['errorFlag'])

    # change from Matlab 1-indexing to Python 0-indexing
    data['time_steps']['traj'] -= 1

    # reshuffle P
    data['time_steps']['P'] = data['time_steps']['P'].reshape(-1, 4, order='F')[:, [0, 2, 3, 1]].reshape(-1,2,2)
    # reshuffle D
    data['time_steps']['D'] = data['time_steps']['D'].reshape(-1, 4, 4, order='F')[:, [[0], [2], [3], [1]], [[0, 2, 3, 1]]].reshape(-1, 2, 2, 2, 2)

    # rename keys
    data['time_steps']['trajectory'] = data['time_steps'].pop('traj')
    if 'bifurcMode' not in data['time_steps']:
        print('!! Warning !! No bifurcation mode data found')
        errors.append([geom, 'No bifurcation mode data found', 12])
    else:
        data['time_steps']['bifurcation_mode'] = data['time_steps'].pop('bifurcMode')
    data['simulations']['error_flag'] = data['simulations'].pop('errorFlag')
    data['time_steps']['is_bifurcation_point'] = data['time_steps'].pop('bifurc')

    if len(data['time_steps']['F']) == 0:
        print('!! Warning !! No simulations found')
        errors.append([geom, 'No simulations found', len(data['simulations']['F_final'])])
        data_all.append([])
        continue

    # reshape/remove trailing dimension
    for key in ['computation_time', 'error_flag']:
        data['simulations'][key] = data['simulations'][key][:, 0]
    data['time_steps']['trajectory'] = data['time_steps']['trajectory'][:, 0]
    data['time_steps']['is_bifurcation_point'] = data['time_steps']['is_bifurcation_point'][:, 0]

    # reshape bifurcation mode
    if 'bifurcation_mode' in data['time_steps']:
        data['time_steps']['bifurcation_mode'] = data['time_steps']['bifurcation_mode'][0] # remove singleton dimension
        for i, mode in enumerate(data['time_steps']['bifurcation_mode']):
            if mode.shape == (0, 0):
                data['time_steps']['bifurcation_mode'][i] = None
            else:
                data['time_steps']['bifurcation_mode'][i] = mode.reshape(-1, 2)

    lengths = [len(data['time_steps'][key]) for key in data['time_steps']]
    if len(set(lengths)) != 1:
        print('!! Warning !! Not all time steps have the same length')
        print(lengths)
        errors.append([geom, 'Missing time steps', max(lengths) - min(lengths)])
        data_all.append([])
        continue
    print('Nr of time steps:', len(data['time_steps']['F']))
    print('Nr of nodes:', data['time_steps']['microfluctuation'].shape[1])

    # ====================== LOAD SPECIAL NODES DATA ======================
    matfile = os.path.join(results_path, geom + '_specialnodes.mat')
    data_from_mat_specialnodes = sio.loadmat(matfile)
    # change from Matlab 1-indexing to Python 0-indexing
    data['mesh']['RVE']['fixed_node'] = data_from_mat_specialnodes['fixed_node'][0,0]-1
    data['mesh']['RVE']['source_nodes'] = data_from_mat_specialnodes['source_nodes'][:, 0]-1
    data['mesh']['RVE']['image_nodes'] = data_from_mat_specialnodes['image_nodes'][:, 0]-1

    # ====================== LOAD GEOMETRY DATA (.mat) ======================
    # load data that was sent to matlab
    path = os.path.join(geometries_path, geom + '.mat')
    data_to_mat = sio.loadmat(path)

    data['mesh']['RVE']['p'] = data_to_mat['p']
    data['mesh']['RVE']['t'] = data_to_mat['t']
    data['mesh']['RVE']['boundary_inds'] = data_to_mat['boundary_inds']
    data['mesh']['RVE']['inds_per_fd'] = data_to_mat['inds_per_fd']
    data['mesh']['RVE']['volume_fraction'] = data_to_mat['volume_fraction'][0,0]

    # check dtype
    if data['mesh']['RVE']['boundary_inds'].dtype == object:
        b = data['mesh']['RVE']['boundary_inds'][0]
        data['mesh']['RVE']['boundary_inds'] = [b2[0].tolist() for b2 in b]
    else:
        data['mesh']['RVE']['boundary_inds'] = data['mesh']['RVE']['boundary_inds'].tolist()

    # ====================== LOAD GEOMETRY DATA (.pkl) ======================
    # load data from .pkl file from geometry generation
    with open(os.path.join(geometries_path, geom + '.pkl'), 'rb') as f:
        data_pkl = pickle.load(f)

    # add unit cell data
    for key, value in data_pkl['uc'].items():
        data['geometry']['unit_cell'][key] = value

    # rename lattice vectors to lattice_vectors
    data['geometry']['unit_cell']['lattice_vectors'] = data['geometry']['unit_cell'].pop('lattice vectors')

    # delete equiv_nodes, is equivalent to periodic_nodes
    del data['geometry']['unit_cell']['equiv_nodes']

    # add fundamental domain data
    for key, value in data_pkl['fd'].items():
        data['geometry']['fundamental_domain'][key] = value

    # delete unnecessary keys
    del data['geometry']['fundamental_domain']['points']
    del data['geometry']['fundamental_domain']['bound_inds']
    del data['geometry']['fundamental_domain']['edges']
    del data['geometry']['fundamental_domain']['n_points']
    del data['geometry']['fundamental_domain']['n_edges']

    # add unique holes data
    data['geometry']['unique_faces'] = data_pkl['unique_holes']

    uf = data['geometry']['unique_faces']
    for elem in uf:
        if 'equiv_holes' in elem:
            elem['equiv_faces'] = elem.pop('equiv_holes')
        if 'all_holes' in elem:
            elem['all_faces'] = elem.pop('all_holes')

    for elem in data['geometry']['unique_faces']:
        # to do: might throw an error if bisectors_x or splint_point_dists are not defined
        if 'bisectors_x' in elem:
            elem['bisectors_x'] = np.array(elem['bisectors_x'])
        if 'spline_point_dists' in elem:
            elem['spline_point_dists'] = np.array(elem['spline_point_dists'])

    data['mesh']['fundamental_domain']['p'] = data_pkl['p']
    data['mesh']['fundamental_domain']['t'] = data_pkl['t']

    data['mesh']['unit_cell']['p'] = data_pkl['p_all']
    data['mesh']['unit_cell']['t'] = data_pkl['t_all']
    data['mesh']['unit_cell']['boundary_inds'] = data_pkl['boundary_inds']

    # ====================== CREATE convenient quantities ======================
    # Calculate position of points at each time step
    F = data['time_steps']['F']  # shape (n_time_steps, 2, 2)
    x_0 = data['mesh']['RVE']['p']  # shape (n_points, 2)
    w = data['time_steps']['microfluctuation']  # shape (n_time_steps, n_points, 2)
    x = np.einsum('ijk,lk->ilj', F, x_0) + w
    data['time_steps']['x'] = x

    # Create edges
    # turn the elements into edges and deduplicate them
    t = data['mesh']['RVE']['t']
    edges = np.vstack((t[:, [0, 3]],
                    t[:, [3, 1]],
                    t[:, [1, 4]],
                    t[:, [4, 2]],
                    t[:, [2, 5]],
                    t[:, [5, 0]]))
    edges2 = np.sort(edges, axis=1)
    edges3, counts = np.unique(edges2, axis=0, return_counts=True)
    data['mesh']['RVE']['edges'] = edges3

    # find all edges that are not shared by two triangles: boundary edges
    # (either boundary of the RVE of boundary of a hole)
    # for each edge in the boundary, check if both nodes are in the same array in boundary_inds
    # if so, it is a unit cell boundary edge
    b_edges = edges3[counts!=2]
    bools = np.zeros(len(b_edges), dtype=bool)
    # print(len(data['mesh']['RVE']['boundary_inds']), len(data['mesh']['RVE']['boundary_inds'][0]))
    for b in data['mesh']['RVE']['boundary_inds']:
        bools[np.isin(b_edges[:, 0], b)*np.isin(b_edges[:, 1], b)] = True
    bools2 = np.zeros(len(edges3), dtype=bool)
    bools2[counts!=2] = bools
    data['mesh']['RVE']['boundary_edges_inds'] = np.where(bools2)[0]

    # select hole boundary edges
    # (edges that are boundary edges but not unit cell boundary edges)
    bools3 = np.zeros(len(edges3), dtype=bool)
    bools3[counts!=2] = True  # all boundary edges
    bools3[bools2] = False     # remove unit cell boundary edges, leaving only hole boundary edges  #!! edge case: if a hole boundary edge is also a unit cell boundary edge, this will go wrong!!
    data['mesh']['RVE']['hole_boundary_edges_inds'] = np.where(bools3)[0]

    # ====================== REMOVE CONTACT ======================
    edges = data['mesh']['RVE']['edges']
    hb_inds = data['mesh']['RVE']['hole_boundary_edges_inds']
    hb_edges = edges[hb_inds]
    # only use coordinates of hole boundary nodes and edges at the hole boundary
    hb_nodes, hb_edges = np.unique(hb_edges, return_inverse=True)
    hb_edges = hb_edges.reshape(-1, 2)
    hb_x = data['time_steps']['x'][:, hb_nodes]

    # specify end and start of each trajectory
    traj = data['time_steps']['trajectory']
    n_trajs = np.max(traj)+1
    start_inds = np.concatenate(([0], np.where(traj[:-1] != traj[1:])[0]+1))
    end_inds = np.concatenate((start_inds[1:], [len(traj)]))

    ends_in_contact = np.zeros(n_trajs, dtype=bool)
    i = 0
    while i < len(traj):
        # print(i)
        traj_i = traj[i]

        # Get the positions of the nodes
        x = data['time_steps']['x'][i]
        hb_x_i = hb_x[i]

        # check if a pair of edges intersects
        cr = hf.crossing_edges(hb_x_i, hb_edges)

        if cr.size > 0:
            ends_in_contact[traj_i] = True
            # print(f'i={i}, traj={traj[i]}, contact detected')

            end_inds[traj_i] = i # this trajectory ends early
            if traj_i == n_trajs-1: # last trajectory, break for loop
                break
            else: # skip rest of the time steps of this trajectory, go to start of next trajectory
                if i > start_inds[traj_i+1]:
                    raise ValueError('i should go back to a previous time step, which makes no sense')
                i = start_inds[traj_i+1]

        else:
            i += 1


    inds_to_keep = np.concatenate([np.arange(i,j) for i,j in zip(start_inds, end_inds)])
    print(f'Trajectory lengths: {end_inds-start_inds}')
    error2 = data['simulations']['error_flag']
    error2[ends_in_contact] = False
    print(f'Error before contact: {error2}')

    if np.any(end_inds - start_inds <= 2):
        print('!! Warning !! One or more trajectories have length <= 2')
        errors.append([geom, 'Trajectories of length <= 2 after removing contact', np.sum(end_inds-start_inds <= 2), np.where(end_inds-start_inds <= 2)[0]])
    if np.any(error2):

        print('!! Warning !! One or more trajectories still end in an error when contact is removed')
        n_err = error2.sum()
        errors.append([geom, 'Error remains after removing contact', n_err, np.where(error2)[0]])

    for key in data['time_steps']:
        data['time_steps'][key] = data['time_steps'][key][inds_to_keep]

    data['simulations']['error_flag'] = error2.astype(bool)
    data['simulations']['ends_in_contact'] = ends_in_contact.astype(bool)

    # sum up is_bifurcation_point per trajectory
    inds_slice = np.nonzero(np.diff(data['time_steps']['trajectory']))[0] + 1
    inds_slice = np.insert(inds_slice, 0, 0)  # add zero at the beginning
    data['simulations']['contains_bifurcation'] = np.add.reduceat(data['time_steps']['is_bifurcation_point'], inds_slice).astype(bool)

    data_all.append(data)


# Convenient quantities

In [ ]:
err = np.array([data['simulations']['error_flag'][0] if 'simulations' in data else True for data in data_all])
print(err)

In [ ]:
# Find the mesh with the lowest hmax (=clmax) value to use as ground truth
hmax_temp = hmax.copy()
hmax_temp[err] = np.inf
ground_truth_i = np.argmin(hmax_temp)
print(ground_truth_i)
print(hmax[ground_truth_i])

In [ ]:
# find geometry closest to the actual hmax (0.040824829046386304)
chosen_i = np.where(np.array(hmax)>0.04)[0][-1]
chosen_i
hmax[chosen_i]

In [ ]:
tmax = data_all[ground_truth_i]['time_steps']['Time'][-1, 0]
tmax

In [ ]:
temp = [
    [
        data['time_steps']['Time'][0, 0],
        data['time_steps']['Time'][-1, 0],
        data['simulations']['ends_in_contact'][0],
        data['simulations']['error_flag'][0],
    ] if 'time_steps' in data else [-1, -1, -1, -1] for data in data_all]
for i, temp_temp in enumerate(temp):
    begin, end, contact, error_flag = temp_temp
    # print(begin, end, contact, error_flag)
    print(f'hmax={hmax[i]:7.3}  {begin:.2f} - {end:.2f}, ends in {"contact"*bool(contact)}{"error"*bool(error_flag)}{"nothing special"*(not contact and not bool(error_flag))}')

In [ ]:
colors = plt.cm.rainbow(np.linspace(0, 1, len(data_all)))

# P

## Plot P

In [ ]:
# plot the evolution of the stress components during trajectory 1
fig, axes = plt.subplots(2,2, figsize=(8,8))
F = data_all[0]['simulations']['F_final']
fig.suptitle(f'At Time=1.0, F=[[{F[0,0]:.3}, {F[0,1]:.3}], [{F[1,0]:.3}, {F[1,1]:.3}]]')

for k, data in enumerate(data_all):
    geom = geoms[k][5:-3]
    Time = data['time_steps']['Time']
    P = data['time_steps']['P']

    for ax, [i,j] in zip(axes.flatten(), [[0,0], [0,1], [1,0], [1,1]]):
        ax.plot(Time, P[:, i, j], label=f'{float(geom):.3}', marker='o', c=colors[k])

    # add a vertical line at each bifurcation point
    t_bif = np.where(data['time_steps']['is_bifurcation_point'])[0]
    for t in t_bif:
        for ax in axes.flatten():
            ax.axvline(Time[t], c=colors[k], linestyle='--') #, label='bifurcation')

for ax, [i,j] in zip(axes.flatten(), [[0,0], [0,1], [1,0], [1,1]]):
    ax.set_ylabel('$P_{' + str(i+1) + str(j+1) + '}$')
    ax.set_xlabel('Time')

# reverse order of legend
handles, labels = ax.get_legend_handles_labels()
fig.legend(handles, labels, loc='center right', title='hmax')

# add more space in between the subplots
plt.tight_layout()
plt.subplots_adjust(top=0.9, bottom=0.1, hspace=0.3, wspace=0.3)

## Error in P

In [ ]:
from scipy.interpolate import interp1d

## Interpolate

In [ ]:
interpolationsP = []
for data in data_all:
    Time = data['time_steps']['Time'][:, 0]
    P = data['time_steps']['P']
    Time2 = np.arange(0, tmax, 0.1)
    spl = interp1d(Time, P, axis=0, kind='linear', fill_value=np.nan, bounds_error=False)

    vals = spl(Time2)
    interpolationsP.append(vals)

In [ ]:
frob_errorP = []
rel_frob_errP = []
frob_gtP = np.linalg.norm(interpolationsP[ground_truth_i], axis=(1,2))	# frobenius norm of ground truth
for vals in interpolationsP:
    frob_errorP.append(np.linalg.norm(vals - interpolationsP[ground_truth_i], axis=(1,2)))
    rel_frob_errP.append(frob_errorP[-1] / frob_gtP
)
frob_errorP

## Plot interpolations

In [ ]:
# plot the evolution of the stress components during trajectory 1
fig, axes = plt.subplots(2,2, figsize=(8,8))
F = data_all[0]['simulations']['F_final']
fig.suptitle(f'At Time=1.0, F=[[{F[0,0]:.3}, {F[0,1]:.3}], [{F[1,0]:.3}, {F[1,1]:.3}]]')

for k, data in enumerate(data_all):
    geom = geoms[k][5:-3]
    Time = data['time_steps']['Time'][:, 0]
    P = data['time_steps']['P']

    for ax, [i,j] in zip(axes.flatten(), [[0,0], [0,1], [1,0], [1,1]]):
        ax.plot(Time2, interpolationsP[k][:, i, j], label=f'{float(geom):.2}', c=colors[k])
        ax.scatter(Time, P[:, i, j], c=colors[[k]], s=20)

    # add a vertical line at each bifurcation point
    t_bif = np.where(data['time_steps']['is_bifurcation_point'])[0]
    for t in t_bif:
        for ax in axes.flatten():
            ax.axvline(Time[t], c=colors[k], linestyle='--')

for ax, [i,j] in zip(axes.flatten(), [[0,0], [0,1], [1,0], [1,1]]):
    ax.set_ylabel('$P_{' + str(i+1) + str(j+1) + '}$')
    ax.set_xlabel('Time')

# reverse order of legend
handles, labels = ax.get_legend_handles_labels()
fig.legend(handles, labels, loc='center right', title='hmax')

# add more space in between the subplots
plt.tight_layout()
plt.subplots_adjust(top=0.9, bottom=0.1, hspace=0.3, wspace=0.3)

## Plot error in P

In [ ]:
plt.figure()
for i, err in enumerate(rel_frob_errP):
    print(i)
    if i == ground_truth_i:
        # add a vertical line at each bifurcation point
        t_bif = np.where(data_all[i]['time_steps']['is_bifurcation_point'])[0]
        print('t_bif:', t_bif)
        for t in t_bif:
            plt.gca().axvline(Time[t], c=colors[i], linestyle='--') #, label='bifurcation')
        continue
    print(hmax[i])
    plt.plot(Time2, err, c=colors[i], label=f'{hmax[i]:.2}')
    if i == chosen_i:
        # plot dashed line
        plt.plot(Time2, err, c='black', linewidth=4, zorder=-1, linestyle='--')

# horizontal line at y=0.05
plt.axhline(0.05, c='black', linestyle='--')
plt.axhline(0.1, c='black', linestyle=':')

plt.xlabel('Time')
plt.ylabel(r'Relative Frobenius error in $\mathbf{P}$')
plt.yscale('log')

# reverse order of legend
handles, labels = plt.gca().get_legend_handles_labels()
plt.legend(handles, labels, loc='center right', title='hmax')


# D

## Plot D

In [ ]:
inds = []
for i in [0, 1]:
    for j in [0, 1]:
        for k in [0, 1]:
            for l in [0, 1]:
                inds.append([i, j, k, l])

In [ ]:
# plot the evolution of the stress components during trajectory 1
fig, axes = plt.subplots(4,4, figsize=(12,12))
F = data_all[0]['simulations']['F_final']
fig.suptitle(f'At Time=1.0, F=[[{F[0,0]:.3}, {F[0,1]:.3}], [{F[1,0]:.3}, {F[1,1]:.3}]]')

for k, data in enumerate(data_all):
    geom = geoms[k][5:-3]
    Time = data['time_steps']['Time']
    D = data['time_steps']['D']

    for ax, [i,j,l,m] in zip(axes.flatten(), inds):
        ax.plot(Time, D[:, i, j, l, m], label=geom, marker='o', c=colors[k])

    # add a vertical line at each bifurcation point
    t_bif = np.where(data['time_steps']['is_bifurcation_point'])[0]
    for t in t_bif:
        for ax in axes.flatten():
            ax.axvline(Time[t], c=colors[k], linestyle='--')

for ax, [i,j,l,m] in zip(axes.flatten(), inds):
    ax.set_ylabel('$D_{' + str(i+1) + str(j+1) + str(l+1) + str(m+1) +'}$')
    ax.set_xlabel('Time')

# reverse order of legend
handles, labels = ax.get_legend_handles_labels()
fig.legend(handles[::-1], labels[::-1], loc='center right', title='hmax')

# add more space in between the subplots
plt.tight_layout()
plt.subplots_adjust(top=0.9, bottom=0.1, hspace=0.5, wspace=0.5)

## Interpolate

In [ ]:
interpolationsD = []
Time2 = np.arange(0, tmax, 0.1)  #0.0005)

for data in data_all:
    Time = data['time_steps']['Time'][:, 0]
    D = data['time_steps']['D']

    spl = interp1d(Time, D, axis=0, kind='linear', fill_value=np.nan, bounds_error=False)
    vals = spl(Time2)
    interpolationsD.append(vals)

## Plot interpolations

In [ ]:
# plot the evolution of the stress components during trajectory 1
fig, axes = plt.subplots(4,4, figsize=(12,12))

F = data_all[0]['simulations']['F_final']
fig.suptitle(f'At Time=1.0, F=[[{F[0,0]:.3}, {F[0,1]:.3}], [{F[1,0]:.3}, {F[1,1]:.3}]]')

for k, data in enumerate(data_all):
    geom = geoms[k][5:-3]
    Time = data['time_steps']['Time']
    D = data['time_steps']['D']

    for ax, [i,j,l,m] in zip(axes.flatten(), inds):
        ax.scatter(Time, D[:, i, j, l, m], label=f'{float(geom):.2}', marker='o', c=[colors[k]], s=10)

    # add a vertical line at each bifurcation point
    t_bif = np.where(data['time_steps']['is_bifurcation_point'])[0]
    for t in t_bif:
        for ax in axes.flatten():
            ax.axvline(Time[t], c=colors[k], linestyle='--') #, label='bifurcation')

    # add interpolated lines
    for ax, [i,j,l,m] in zip(axes.flatten(), inds):
        ax.plot(Time2, interpolationsD[k][:, i, j, l, m], c=colors[k])

for ax, [i,j,l,m] in zip(axes.flatten(), inds):
    ax.set_ylabel('$D_{' + str(i+1) + str(j+1) + str(l+1) + str(m+1) +'}$')
    ax.set_xlabel('Time')

# reverse order of legend
handles, labels = ax.get_legend_handles_labels()
fig.legend(handles[::-1], labels[::-1], loc='center right', title='hmax')

# add more space in between the subplots
plt.tight_layout()
plt.subplots_adjust(top=0.9, bottom=0.1, hspace=0.5, wspace=0.5)

In [ ]:
# plot the evolution of only D_1111
fig, ax = plt.subplots(figsize=(12,12))

F = data_all[0]['simulations']['F_final']
fig.suptitle(f'At Time=1.0, F=[[{F[0,0]:.3}, {F[0,1]:.3}], [{F[1,0]:.3}, {F[1,1]:.3}]]')

for k, data in enumerate(data_all):
    geom = geoms[k][5:-3]
    Time = data['time_steps']['Time']
    D = data['time_steps']['D']

    ax.scatter(Time, D[:, 0, 0, 0, 0], label=f'{float(geom):.2}', marker='o', c=[colors[k]], s=10)

    # add a vertical line at each bifurcation point
    t_bif = np.where(data['time_steps']['is_bifurcation_point'])[0]
    for t in t_bif:
        ax.axvline(Time[t], c=colors[k], linestyle='--') #, label='bifurcation')

    # add interpolated lines
    ax.plot(Time2, interpolationsD[k][:, 0, 0, 0, 0], c=colors[k])

ax.set_ylabel('$D_{' + str(i+1) + str(j+1) + str(l+1) + str(m+1) +'}$')
ax.set_xlabel('Time')

# reverse order of legend
handles, labels = ax.get_legend_handles_labels()
fig.legend(handles, labels, loc='center right', title='hmax')

# add more space in between the subplots
plt.tight_layout()
plt.subplots_adjust(top=0.9, bottom=0.1, hspace=0.5, wspace=0.5)

## Calculate and plot error

In [ ]:
frob_gtD = np.linalg.norm(data_all[ground_truth_i]['time_steps']['D'][0].reshape(-1, 16), axis=-1)	# frobenius norm of ground truth, first time step only

frob_errorD = []
rel_frob_errD = []
for vals in interpolationsD:
    frob_errorD.append(np.linalg.norm((vals - interpolationsD[ground_truth_i]).reshape(-1, 16), axis=-1))
    rel_frob_errD.append(frob_errorD[-1] / frob_gtD)
frob_errorD

In [ ]:
# %matplotlib qt
plt.figure()
for i, err in enumerate(rel_frob_errD):
    if i == ground_truth_i:
        # add a vertical line at each bifurcation point
        t_bif = np.where(data_all[i]['time_steps']['is_bifurcation_point'])[0]
        print('t_bif:', t_bif)
        for t in t_bif:
            plt.gca().axvline(Time[t], c=colors[i], linestyle='--')
        continue

    if i == chosen_i:
        # add a vertical line at each bifurcation point
        t_bif = np.where(data_all[i]['time_steps']['is_bifurcation_point'])[0]
        print('t_bif:', t_bif)
        for t in t_bif:
            plt.gca().axvline(Time[t], c=colors[i], linestyle='--')

    print(hmax[i])
    plt.plot(Time2, err, c=colors[i], label=f'{float(hmax[i]):.2}')
    if i == chosen_i:
        # plot dashed line
        plt.plot(Time2, err, c='black', linewidth=4, zorder=-1, linestyle='--')

# horizontal line at y=0.05
plt.axhline(0.05, c='black', linestyle='--')
plt.axhline(0.1, c='black', linestyle=':')

# plt.ylim([0, 0.25])
plt.xlabel('Time')
plt.yscale('log')
plt.ylabel('Relative Frobenius error in $\mathbf{D}$')

# reverse order of legend
handles, labels = plt.gca().get_legend_handles_labels()
plt.legend(handles, labels, loc='center right', title='hmax')


# Plot P, D and deformation

In [ ]:
plt.rcParams.update({
    "text.usetex": True,
    'text.latex.preamble': r'\usepackage{{amsmath}} \usepackage{{amssymb}} \usepackage{{xcolor}}',
    'font.size': 14
})

In [ ]:
fig, ax = plt.subplots(figsize=(6,3.5))

# Plot error in P
for i, err in enumerate(rel_frob_errP):
    print(i)
    if i == ground_truth_i:
        # add a vertical line at each bifurcation point
        t_bif = np.where(data_all[i]['time_steps']['is_bifurcation_point'])[0]
        print('t_bif:', t_bif)
        for t in t_bif:
            ax.axvline(Time[t], c=colors[i], linestyle='--') #, label='bifurcation')
        continue
    print(hmax[i])
    ax.plot(Time2, err, c=colors[i], label=f'{hmax[i]:.2}', linestyle='--', marker='o')
    if i == chosen_i:
        # plot dashed line
        ax.plot(Time2, err, c='black', linewidth=4, zorder=-1, )

# horizontal line at y=0.05
ax.axhline(0.05, c='black', linestyle='--')
ax.axhline(0.1, c='black', linestyle=':')

ax.set_xlabel(r'$\tau$')
ax.set_ylabel(r'$\epsilon_{P}$')
ax.set_yscale('log')

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, labels, loc='center right', title=r'$cl_{\textrm{max}}$',
          bbox_to_anchor=(1.45, 0.5))

plt.subplots_adjust(left=0.2, right=0.7, bottom=0.2, top=0.9)

fig.savefig(f'{geom_base}_mesh_convergence_P.pdf', bbox_inches='tight')



In [ ]:
fig, ax = plt.subplots(figsize=(6,3.5))

# Plot error in D
for i, err in enumerate(rel_frob_errD):
    if i == ground_truth_i:
        # add a vertical line at each bifurcation point
        t_bif = np.where(data_all[i]['time_steps']['is_bifurcation_point'])[0]
        print('t_bif:', t_bif)
        for t in t_bif:
            ax.axvline(Time[t], c=colors[i], linestyle='--')
        continue

    if i == chosen_i:
        # add a vertical line at each bifurcation point
        t_bif = np.where(data_all[i]['time_steps']['is_bifurcation_point'])[0]
        print('t_bif:', t_bif)
        for t in t_bif:
            ax.axvline(Time[t], c=colors[i], linestyle='--')

    print(hmax[i])
    ax.plot(Time2, err, c=colors[i], label=f'{float(hmax[i]):.2}', linestyle='--', marker='o')
    if i == chosen_i:
        # plot different style line
        ax.plot(Time2, err, c='black', linewidth=4, zorder=-1, )

# horizontal line at y=0.05
ax.axhline(0.05, c='black', linestyle='--')
ax.axhline(0.1, c='black', linestyle=':')

# ax.ylim([0, 0.25])
ax.set_xlabel(r'$\tau$')
ax.set_yscale('log')
ax.set_ylabel(r'$\epsilon_{D}$')

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, labels, loc='center right', title=r'$cl_{\textrm{max}}$',
          bbox_to_anchor=(1.45, 0.5))

plt.subplots_adjust(left=0.2, right=0.7, bottom=0.2, top=0.9)
fig.savefig(f'{geom_base}_mesh_convergence_D.pdf', bbox_inches='tight')


In [ ]:
# plot the mesh, before and after deformation
fig, ax = plt.subplots(figsize=(6,6))

p = data_all[ground_truth_i]['mesh']['RVE']['p'] # initial node positions
t = data_all[ground_truth_i]['mesh']['RVE']['t'] # initial node positions
e = data_all[ground_truth_i]['mesh']['RVE']['edges'] # mesh edges
hb_inds = data_all[ground_truth_i]['mesh']['RVE']['hole_boundary_edges_inds'] # indices of the hole boundary edges

lv = data_all[ground_truth_i]['geometry']['unit_cell']['lattice_vectors']

# deformed position at the last time step of the trajectory
p_def = data_all[ground_truth_i]['time_steps']['x'][-1]
F_temp = data_all[ground_truth_i]['time_steps']['F'][-1]
lv_def = np.dot(F_temp, lv.T).T

# plot elements of the original mesh, 2x2 RVE
for shifts in [[0, 0], [1, 0], [0, 1], [1, 1]]:
    p2 = p +2*shifts[0]*lv[0] +2*shifts[1]*lv[1]

    # plot hole boundary edges of deformed mesh
    x, y = np.transpose(p2[e[hb_inds].T], axes=[2,0,1])
    c='lightgrey'
    edges0 = ax.plot(x, y, c=c, linewidth=1)

# plot elements and hole boundaries of the deformed mesh, 2x2 RVE
for shifts in [[0, 0], [1, 0], [0, 1], [1, 1]]:
    p_def2 = p_def +2*shifts[0]*lv_def[0] +2*shifts[1]*lv_def[1]

    c = 'darkblue'
    x, y = np.transpose(p_def2[e[hb_inds].T], axes=[2,0,1])
    edges0 = ax.plot(x, y, c=c, linewidth=1)

# set aspect ratio to be equal
ax.set_aspect('equal')

# ax.gca().set_title(f'F={F_final}')
temp_str = r'$\textbf{\textrm{F}}=\begin{pmatrix}' + f'{F_temp[0,0]:.2}' + ' & ' + f'{F_temp[0,1]:.2}' + r'\\' + f'{F_temp[1,0]:.2}' + ' & ' + f'{F_temp[1,1]:.2}' + r'\end{pmatrix}$'
if data_all[ground_truth_i]['simulations']['error_flag'][-1]:
    temp_str = temp_str + r', error_flag!'
    # make latex textcolor red
    temp_str = r'\textcolor{red}{' + temp_str + '}'

ax.set_title(temp_str)

# fig.subplots_adjust(wspace=0.3, hspace=0.3)
fig.savefig(f'{geom_base}_mesh_convergence_geometry.pdf', bbox_inches='tight')
